In [262]:
import pandas as pd
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go

#chartTemplate = 'plotly_dark'
chartTemplate = 'plotly_white'

In [318]:
cases = pd.read_csv("covid19-casos.csv")

cases_cfg = cases[cases["provincia"] == "Cienfuegos"]

cuba = cases['fecha_confirmacion'].value_counts().reset_index().rename(columns={'index': 'Date','fecha_confirmacion':'N'}).sort_values(by='Date', ascending=True)
cfg = cases_cfg['fecha_confirmacion'].value_counts().reset_index().rename(columns={'index': 'Date','fecha_confirmacion':'N'}).sort_values(by='Date', ascending=True)

# calculate case velocity
#cuba["vel"] = cuba.loc[:, cuba.columns != 'Date'].diff()
cuba["vel"] = cuba.loc[:, 'N'].diff()
cfg["vel"] = cfg.loc[:, 'N'].diff()

# calculate case acceleration
cuba["acc"] = cuba[["Date", "vel"]].loc[:, 'vel'].diff()
cfg["acc"] = cfg[["Date", "vel"]].loc[:, 'vel'].diff()

# calculate means - 14 días - 2 semanas
# usando rolling
avg = 14
cfg["accelMeans14"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = avg).mean()
cfg["accelMeans7"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = 3).mean()
cuba["accelMeans14"] = cuba[["Date", "vel"]].loc[:, 'vel'].rolling(window = avg).mean()
cuba["accelMeans7"] = cuba[["Date", "vel"]].loc[:, "vel"].rolling(window = 3).mean()

# con EWM
# cfg["accelMeans5"] = cfg[["Date", "vel"]].loc[:, cfg[["Date", "vel"]].columns != 'Datetime'].ewm(span=5).mean()

cuba

C:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning:

Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.



,Date,N,vel,acc,accelMeans14,accelMeans7
459,2020/03/11,3,NaN,NaN,NaN,NaN
473,2020/03/13,1,-2.0,NaN,NaN,NaN
476,2020/03/16,1,0.0,2.0,NaN,NaN
466,2020/03/17,2,1.0,1.0,NaN,-0.333333
457,2020/03/18,4,2.0,1.0,NaN,1.000000
...,...,...,...,...,...,...
6,2021/06/30,2952,-18.0,92.0,109.571429,121.000000
2,2021/07/01,3308,356.0,374.0,130.500000,76.000000
1,2021/07/02,3475,167.0,-189.0,143.071429,168.333333
0,2021/07/03,3519,44.0,-123.0,148.785714,189.000000


In [295]:
fig = px.line(cfg, x="Date", y="N", template=chartTemplate)
fig.update_layout({"title": 'Casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [296]:
fig = px.line(cfg, x="Date", y="vel", template=chartTemplate)

fig.update_layout({"title": 'Velocidad de casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"}})
                   #"showlegend": False})
ffig.show()

In [297]:
cfg.head(5)

,Date,N,vel,acc,accelMeans14
231,2020/03/24,1,NaN,NaN,NaN
195,2020/03/27,3,2.0,NaN,NaN
217,2020/03/28,1,-2.0,-4.0,NaN
200,2020/03/29,3,2.0,4.0,NaN
197,2020/04/01,3,0.0,-2.0,NaN


In [320]:
#import numpy as np
# here you define the color
#cfg["color"] = np.where(cfg["accelMeans14"] >= 0, 'green', "red")
## add a color column to the df, apply along row
#cfg['color'] = cfg.apply(lambda x: 'red' if x['accelMeans14'] >= 0 else 'green', axis=1)
pos = cfg[cfg['accelMeans7'] >= 0]
neg = cfg[cfg['accelMeans7'] < 0]

fig = go.Figure()

fig = px.line(cfg, x="Date", y="accelMeans14", template=chartTemplate)

#fig.add_trace(go.Bar(x=cfg['Date'], y=cfg['accelMeans14'], name='Promedio de 14 días', marker = dict(color = 'red')) 

fig.add_trace(go.Bar(x=pos['Date'], y=pos['accelMeans7'], name='Promedio de 14 días', marker = dict(color = 'red'), opacity=0.5))
fig.add_trace(go.Bar(x=neg['Date'], y=neg['accelMeans7'], name='Promedio de 14 días', marker = dict(color = 'green'), opacity=0.5))#fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},

                   #"width" : 1000,
                   #"height" : 800,                   "showlegend": False})
fig.show()

#  ##--

# CUBA

## Análisis de Cuba

#  ##--

In [299]:
fig = px.line(cuba, x="Date", y="N", template=chartTemplate)

fig.update_layout({"title": 'Casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [300]:
fig = px.line(cuba, x="Date", y="vel", template=chartTemplate)

fig.update_layout({"title": 'Velocidad - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [301]:
fig = px.line(cuba, x="Date", y="acc", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [326]:
pos = cuba[cuba['accelMeans7'] >= 0]
neg = cuba[cuba['accelMeans7'] < 0]

fig = px.line(cuba, x="Date", y="accelMeans14", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})
fig.add_trace(go.Bar(x=pos['Date'], y=pos['accelMeans7'], name='Promedio de 14 días', opacity=0.5))
fig.add_trace(go.Bar(x=neg['Date'], y=neg['accelMeans7'], name='Promedio de 14 días', opacity=0.5))

fig.show()